In [5]:
import agate

In [6]:
exonerations = agate.Table.from_csv('exonerations-20150828.csv')

print exonerations

|----------------------+---------------|
|  column_names        | column_types  |
|----------------------+---------------|
|  last_name           | Text          |
|  first_name          | Text          |
|  age                 | Number        |
|  race                | Text          |
|  state               | Text          |
|  tags                | Text          |
|  crime               | Text          |
|  sentence            | Text          |
|  convicted           | Number        |
|  exonerated          | Number        |
|  dna                 | Boolean       |
|  dna_essential       | Text          |
|  mistake_witness     | Boolean       |
|  false_confession    | Boolean       |
|  perjury             | Boolean       |
|  false_evidence      | Boolean       |
|  official_misconduct | Boolean       |
|  inadequate_defense  | Boolean       |
|----------------------+---------------|



In [7]:
num_false_confessions = exonerations.aggregate(agate.Count('false_confession', True))

print(num_false_confessions)

211


In [8]:
median_age = exonerations.aggregate(agate.Median('age'))

print(median_age)

26


c:\python27\lib\site-packages\agate\warns.py:19: NullCalculationWarning: Column "age" contains nulls. These will be excluded from Median calculation.
  ), NullCalculationWarning)


In [9]:
num_without_age = exonerations.aggregate(agate.Count('age', None))

print(num_without_age)

9


In [10]:
mass = exonerations.aggregate(agate.Count('state', 'MA'))

print(mass)

41


In [11]:
with_age = exonerations.where(lambda row: row['age'] is not None)

In [12]:
old = len(exonerations.rows)
new = len(with_age.rows)
print old, new
print old - new

1651 1642
9


In [20]:
with_age_MO = exonerations.where(lambda row: row['state'] == 'MO')
print len(with_age_MO.rows)
median_age_MO = with_age_MO.aggregate(agate.Median('age'))
print(median_age_MO)

34
24.5


In [14]:
median_age = with_age.aggregate(agate.Median('age'))

print(median_age)

26


In [15]:
with_years_in_prison = exonerations.compute([
    ('years_in_prison', agate.Change('convicted', 'exonerated'))
])

median_years = with_years_in_prison.aggregate(agate.Median('years_in_prison'))

print(median_years)

8


In [16]:
with_computations = exonerations.compute([
    ('full_name', agate.Formula(agate.Text(), lambda row: '%(first_name)s %(last_name)s' % row)),
    ('years_in_prison', agate.Change('convicted', 'exonerated'))
])

In [17]:
sorted_by_age = exonerations.order_by('age')
youngest_ten = sorted_by_age.limit(10)
youngest_ten.print_table(max_columns=10)

|------------+------------+-----+-----------+-------+---------+---------+----------------+-----------+------------+------|
|  last_name | first_name | age | race      | state | tags    | crime   | sentence       | convicted | exonerated | ...  |
|------------+------------+-----+-----------+-------+---------+---------+----------------+-----------+------------+------|
|  Murray    | Lacresha   |  11 | Black     | TX    | CV, F   | Murder  | 20 years       |     1,996 |      2,001 | ...  |
|  Adams     | Johnathan  |  12 | Caucasian | GA    | CV, P   | Murder  | 1 year         |     2,005 |      2,006 | ...  |
|  Harris    | Anthony    |  12 | Black     | OH    | CV      | Murder  | 7 years        |     1,999 |      2,000 | ...  |
|  Edmonds   | Tyler      |  13 | Caucasian | MS    |         | Murder  | Life           |     2,004 |      2,008 | ...  |
|  Handley   | Zachary    |  13 | Caucasian | PA    | A, CV   | Arson   | 6 to 12 months |     2,008 |      2,015 | ...  |
|  Jimenez   | T

In [18]:
binned_ages = exonerations.bins('age', 10, 0, 100)
binned_ages.print_bars('age', 'Count', width=80)

AttributeError: 'NoneType' object has no attribute 'number_symbols'

In [ ]:
by_state = exonerations.group_by('state')

state_totals = by_state.aggregate([
    ('count', agate.Count())
])

sorted_totals = state_totals.order_by('count', reverse=True)

sorted_totals.print_table(max_rows=5)

In [ ]:
with_years_in_prison = exonerations.compute([
    ('years_in_prison', agate.Change('convicted', 'exonerated'))
])

state_totals = with_years_in_prison.group_by('state')

medians = state_totals.aggregate([
    ('count', agate.Count()),
    ('median_years_in_prison', agate.Median('years_in_prison'))
])

sorted_medians = medians.order_by('median_years_in_prison', reverse=True)

sorted_medians.print_table(max_rows=5)